In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pulp import *
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser
from src.preprocessing.preprocessor import Preprocessor
from src.solvers.solution import Solution
from src.solvers.greedy import GreedySolver
from src.solvers.pulp_solver import PuLPSolver

from time import time
from tqdm import tqdm
from typing import Dict

/tmp/ipykernel_374912/3561605614.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
def get_num_triples(dataset : Dict):
    num_triples= 0
    for n in dataset.keys():
        num_triples += len(dataset[n])
    
    return num_triples

## Test 04

In [4]:
parser = Parser("data/testfiles/test4.txt")

In [5]:
t0 = time()
info = parser.read()
t1 = time()

print(f"Total execution time {t1 - t0} s")

3869it [00:00, 18033.05it/s]

76804it [00:01, 46535.36it/s]


Total execution time 5.859124660491943 s


In [6]:
dataset : Dict[int, pd.DataFrame ]= info['data']
p, K, M, N = info['p'], info['K'], info['M'], info['N']

In [7]:
preprocessing_results = {}
preprocessor = Preprocessor(
    K, 
    M,
    N,
    p,
    dataset
)

t0 = time()

data_1 = preprocessor.remove_trivial_values()
t1 = time()

preprocessing_results['remove trivial values'] = {
    "runtime": (t1 - t0),
    "num_triples": get_num_triples(data_1)
}
t1 = time()

data_2 = preprocessor.remove_ip_dominated(dataset)
t2 = time()
preprocessing_results['remove IP dominated'] = {
    "runtime": t2-t1,
    "num_triples": get_num_triples(data_2)
}
t2 = time()

data_3 = preprocessor.remove_lp_dominated(data_2)
t3 = time()
preprocessing_results['remove LP dominated'] = {
    "runtime": t3-t2,
    "num_triples": get_num_triples(data_3)
}


100%|██████████| 640/640 [00:05<00:00, 108.73it/s]


In [8]:
pd.DataFrame(preprocessing_results)

,remove trivial values,remove IP dominated,remove LP dominated
runtime,16.766417,184.780019,20.072942
num_triples,614400.000000,14688.000000,4974.000000


## Greedy algorithm

In [9]:
lp_results = {}

In [10]:
pulp_solver = PuLPSolver(
    K, 
    M,
    N,
    p,
    data_3
)
pulp_solver.set_problem(data_3)

t1 = time()
pulp_solver.solve()
t2 = time()

lp_results['pulp'] = {
    'runtime': t2-t1,
    'data_rate' : pulp_solver.solution.objective.value()
}

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/pedro/anaconda3/envs/user-scheduling-in-5g/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0b52388c47ad4d9e99198503ca1d5556-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/0b52388c47ad4d9e99198503ca1d5556-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 646 COLUMNS
At line 15569 RHS
At line 16211 BOUNDS
At line 21186 ENDATA
Problem MODEL has 641 rows, 4974 columns and 9948 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 641 (0) rows, 4974 (0) columns and 9948 (0) elements
Perturbing problem by 0.001% of 343.39966 - largest nonzero change 0.00056729545 ( 0.0045587318%) - largest zero change 0
0  Obj -0 Primal inf 979.79416 (640) Dual inf 251358.04 (4974)
87  Obj 11382.886 Primal inf 499.25593 (332)
174  Obj 10955.294 Primal inf 341.29992 (254)
261  Obj 10663.951

In [11]:
solver = GreedySolver(
    K, 
    M,
    N,
    p,
    data_3
)

t1 = time()
solver.solve(data_3)
t2 = time()

lp_results['greedy'] = {
    'runtime': t2-t1,
    'data_rate' : solver.solution.get_data_rate()
}


In [12]:
pd.DataFrame(lp_results)

,pulp,greedy
runtime,0.079182,1.170413
data_rate,9870.321839,9870.321839
